In [1]:
import cv2
import numpy as np
import math

cap = cv2.VideoCapture(0)

x0, y0, x1, y1 = 300, 100, 600, 400              
min_hand_area   = 2500                           
defect_depth_px = 20                             


In [2]:

while True:
    ok, frame = cap.read()
    if not ok:
        break
    frame = cv2.flip(frame, 1)
    draw  = frame.copy()

    # 1) ROI crop
    roi_bgr = frame[y0:y1, x0:x1]
    roi_hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)


    lower = np.array([0,  30,  60])   
    upper = np.array([20, 170, 255])
    mask  = cv2.inRange(roi_hsv, lower, upper)

    mask = cv2.medianBlur(mask, 5)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,
                            cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), 2)

 
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    fingers = 0
    if cnts:
        hand = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(hand) > min_hand_area:
            cv2.drawContours(roi_bgr, [hand], -1, (0,255,0), 2)

            # convex hull & defects
            hull_idx = cv2.convexHull(hand, returnPoints=False)
            if hull_idx is not None and len(hull_idx) > 3:
                defects = cv2.convexityDefects(hand, hull_idx)
                if defects is not None:
                    for i in range(defects.shape[0]):
                        s,e,f,d = defects[i,0]
                        start = tuple(hand[s][0])
                        end   = tuple(hand[e][0])
                        far   = tuple(hand[f][0])
      
                        a = np.linalg.norm(np.array(end) - np.array(start))
                        b = np.linalg.norm(np.array(far) - np.array(start))
                        c = np.linalg.norm(np.array(end) - np.array(far))
                        # angle at the far point
                        if b*c == 0: continue
                        angle = np.degrees(np.arccos((b**2 + c**2 - a**2) / (2*b*c)))
                        depth = d / 256.0                   

                        if angle < 90 and depth > defect_depth_px:
                            fingers += 1
                            cv2.circle(roi_bgr, far, 6, (255,0,0), -1)

                    if fingers > 0:
                        fingers += 1          
                    else:
   
                        x, y, w, h = cv2.boundingRect(hand)
                        if h / float(w) > 1.3:  
                            fingers = 1          
                    
    cv2.rectangle(draw, (x0,y0), (x1,y1), (0,255,0), 2)      # ROI
    cv2.putText(draw, str(fingers), (40,80),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 4)
    cv2.imshow("Finger Counter", draw)
    cv2.imshow("Mask", mask)

    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

cap.release()
cv2.destroyAllWindows()
